In [1]:
import pandas as pd

# <font color = 'gold'> Steps to go on in this part of the automation: </font>

<font color = 'teal'>

- Filter the itens that were used in the last 24 months
- Filter all the itens that the status is "ativo"
- Make sure that duplicated itens is no more

</font>

In [5]:
clean_system_legacy = pd.read_excel(io = 'files/itens_to_clean.xlsx', sheet_name=0)

In [7]:
clean_system_legacy.head()

,cod_pro,desc_cadastro,movimentacao_12,movimentacao_24,referencia,saldo,codigo_grupo,desc_grupo,custo_medio,custo_atual,...,simpro,brasindice,cod_bras_conc,tuss,fornecedor,fabricante_medicamento,padronizado,consignado,fabricante,situacao_produto
0,2,AGULHA DESCARTAVEL 40X12,SIM,SIM,NaN,1824.0,2,MATERIAL HOSPITALAR,0.12,0.12,...,260942.0,NaN,NaN,78420130,000198-MACROMED C.M. MED .HOSP. LTDA,NaN,S,N,SOL-MILLEN,ATIVO
1,3,CATETER ELETRODO ESTIM BIP TEMPORARIO 5FR 10X1...,SIM,SIM,NaN,2.0,19,MATERIAL ESPECIAL,330.46,356.60,...,145197.0,NaN,NaN,78231060,000330-BIOMEDICAL PRODUTOS CIENTIFICOS MEDICOS,NEUROMEDEX | ALEMANHA,S,N,BIO MINAS,ATIVO
2,5,CABO ELETRODO ENDOCARDIO TEMPORARIO 5FRX110CM ...,NÃO,NÃO,NaN,0.0,19,MATERIAL ESPECIAL,1.00,0.00,...,21141.0,NaN,NaN,,NaN,ST.JUDE MEDICAL | EUA,S,N,ST.JUDE,ATIVO
3,6,CARNES | FILE DE FRANGO,NÃO,SIM,NaN,0.0,25,FRIGORIFICADO,18.58,19.80,...,NaN,NaN,NaN,NaN,001285-EMPORIO MEGA 100 COM. DE ALIMENTOS S.A.,NaN,S,N,NaN,ATIVO
4,7,DIETA ENTERAL GLUCERNA RTH 1.5 (500ML),NÃO,NÃO,NaN,0.0,8,PREPAROS P/ NUTRICAO,34.76,34.80,...,NaN,NaN,NaN,NaN,000887-NATBIO IMPORTADORA LTDA,NaN,S,N,NaN,INATIVO


In [6]:
clean_system_legacy.movimentacao_24.value_counts()

NÃO    16562
SIM     4794
Name: movimentacao_24, dtype: int64

In [8]:
clean_system_legacy.situacao_produto.value_counts()

ATIVO      19306
INATIVO     2050
Name: situacao_produto, dtype: int64

In [49]:
def filter_itens(df, column_movimentation, column_status):
    # applying the first and second step in the header
    assert (df[column_movimentation].nunique() == 2) and (df[column_status].nunique() == 2)

    df[column_movimentation] = df[column_movimentation].str.upper()
    df[column_status] = df[column_status].str.upper()

    f = (df[column_movimentation] == 'SIM') & (df[column_status] == 'ATIVO')

    return df.loc[f]

In [52]:
def removing_duplicates(df, column_to_clean):
    values = df.groupby(column_to_clean).cumcount() + 1
    
    df['row_number'] = values

    return df.loc[df.row_number == 1]

In [53]:
clean_system_legacy_v2 = filter_itens(clean_system_legacy, column_movimentation = 'movimentacao_24', column_status = 'situacao_produto')
clean_system_legacy_v2 = removing_duplicates(clean_system_legacy_v2, column_to_clean = 'desc_cadastro')

/var/folders/mq/g6k1b1h96tsb47xpdljnctsc0000gp/T/ipykernel_61750/3485347810.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['row_number'] = values


In [54]:
clean_system_legacy_v2

,cod_pro,desc_cadastro,movimentacao_12,movimentacao_24,referencia,saldo,codigo_grupo,desc_grupo,custo_medio,custo_atual,...,brasindice,cod_bras_conc,tuss,fornecedor,fabricante_medicamento,padronizado,consignado,fabricante,situacao_produto,row_number
0,2,AGULHA DESCARTAVEL 40X12,SIM,SIM,NaN,1824.0,2,MATERIAL HOSPITALAR,0.12,0.12,...,NaN,NaN,78420130,000198-MACROMED C.M. MED .HOSP. LTDA,NaN,S,N,SOL-MILLEN,ATIVO,1
1,3,CATETER ELETRODO ESTIM BIP TEMPORARIO 5FR 10X1...,SIM,SIM,NaN,2.0,19,MATERIAL ESPECIAL,330.46,356.60,...,NaN,NaN,78231060,000330-BIOMEDICAL PRODUTOS CIENTIFICOS MEDICOS,NEUROMEDEX | ALEMANHA,S,N,BIO MINAS,ATIVO,1
3,6,CARNES | FILE DE FRANGO,NÃO,SIM,NaN,0.0,25,FRIGORIFICADO,18.58,19.80,...,NaN,NaN,NaN,001285-EMPORIO MEGA 100 COM. DE ALIMENTOS S.A.,NaN,S,N,NaN,ATIVO,1
9,12,DIETA ENTERAL NUTRISON ADVANCED DIASON ENERGY ...,SIM,SIM,NaN,0.0,8,PREPAROS P/ NUTRICAO,40.40,2075.00,...,NaN,NaN,NaN,000272-NUTRIPORT COMERCIAL LTDA,NaN,S,N,NaN,ATIVO,1
10,13,DIETA ENTERAL NUTRISON ENERGY MULTI FIBER 1.5 ...,SIM,SIM,NaN,0.0,8,PREPAROS P/ NUTRICAO,37.27,1235.00,...,NaN,NaN,NaN,000272-NUTRIPORT COMERCIAL LTDA,NaN,S,N,NaN,ATIVO,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21200,21085,PINO OSSEO 4MMX150MM ROSCA 3.3MM-4MMX45MM\t,SIM,SIM,NaN,0.0,24,OSTEOSSINTESE,1200.00,1200.00,...,NaN,NaN,NaN,NaN,LUIZ GUILHERME SARTORI &,S,N,NaN,ATIVO,1
21201,21086,MYRBETRIC 50MG; CP REV LIB PROL,SIM,SIM,NaN,7.0,1,MEDICAMENTOS,10.45,10.45,...,NaN,NaN,NaN,007161-DROGARIA LM LTDA,MYRBETRIC 50MG CP REV LIB,S,N,NaN,ATIVO,1
21202,21087,DAFORIN; 20MG CP REV,SIM,SIM,NaN,26.0,1,MEDICAMENTOS,2.94,2.94,...,NaN,NaN,NaN,005146-DROGARIA SOARES LTDA,EMS SIGMA PHARMA LTDA\t,S,N,NaN,ATIVO,1
21203,21088,CLOPIDOGREL 75MG; CP REV ACCORD,SIM,SIM,NaN,54.0,1,MEDICAMENTOS,6.76,6.76,...,79531.0,0748.19442.FEGF,90429290,NaN,ACCORD FARMACEUTICA LTDA\t,S,N,ACCORD,ATIVO,1
